The contractor passed the rug along to a neighbor. 
Here's the important info for finding them:

> I gave it to this guy who lived in my neighborhood... he was a Cancer born in the year of the Rabbit...


So I'll need to find the customer who:
- FIXME: Update for speedrun as well
- lives in Joshua Peterson's neighborhood
- is a Cancer: born between the dates of June 21st and July 22nd ([source](https://www.horoscope.com/zodiac-signs/cancer))
- was born in the year of the Rabbit: any of the following years: 2023, 2011, 1999, 1987, 1975, 1963, and 1951 ([source](https://chinesenewyear.net/zodiac/rabbit/)) 

In [1]:
import polars as pl
import datetime as dt

customers = pl.read_csv("harder-data/noahs-customers.csv",try_parse_dates=True)

In [15]:
# Phone number is from part 2
# FIXME: Carry it over programatically!
part2_customer_citystatezip = (
    customers.filter(pl.col("phone") == "838-351-0370").select("citystatezip").item()
)
print(part2_customer_citystatezip)
neighbors = customers.filter(pl.col("citystatezip") == part2_customer_citystatezip)
print(neighbors.select("customerid", "name","birthdate"))

Queens Village, NY 11427
shape: (22, 3)
┌────────────┬──────────────────────┬────────────┐
│ customerid ┆ name                 ┆ birthdate  │
│ ---        ┆ ---                  ┆ ---        │
│ i64        ┆ str                  ┆ date       │
╞════════════╪══════════════════════╪════════════╡
│ 1154       ┆ Vincent Walker       ┆ 1981-06-12 │
│ 2550       ┆ Judith Costa         ┆ 1982-03-18 │
│ 3864       ┆ David Perez          ┆ 1979-09-29 │
│ 4721       ┆ Billy Schneider      ┆ 1954-08-03 │
│ 5116       ┆ Rebecca Brenda Baker ┆ 1970-12-03 │
│ …          ┆ …                    ┆ …          │
│ 9244       ┆ Samuel Wilson        ┆ 1966-07-02 │
│ 9461       ┆ Rebecca Cox          ┆ 1963-02-01 │
│ 9921       ┆ Kathleen Gomez       ┆ 1989-07-13 │
│ 10960      ┆ Leonard Jones        ┆ 1992-08-28 │
│ 10966      ┆ Brian Henderson      ┆ 1978-08-28 │
└────────────┴──────────────────────┴────────────┘


In [18]:
# # Cancer: June 21 - July 22
# june_21_through_31 = (
#     pl.col("birthdate").dt.month() == 6,
#     pl.col("birthdate").dt.day().is_in(list(range(21, 32))),
# )
# july_1_through_22 = (
#     pl.col("birthdate").dt.month() == 7,
#     pl.col("birthdate").dt.day().is_in(list(range(1, 23))),
# )

# Libra:  Sep 23 - Oct 22
sept_21_through_30 = (
    pl.col("birthdate").dt.month() == 9,
    pl.col("birthdate").dt.day().is_in(list(range(21, 31))),
)
oct_1_through_22 =    (
    pl.col("birthdate").dt.month() == 10,
    pl.col("birthdate").dt.day().is_in(list(range(1, 23))),
)


# I'd love to learn out how to OR these two filters together instead of concat
neighbors_same_star_sign = pl.concat(
    [neighbors.filter(sept_21_through_30), neighbors.filter(oct_1_through_22)], how="vertical"
)
print(neighbors_same_star_sign.select("customerid", "name", "phone", "birthdate"))

shape: (1, 4)
┌────────────┬─────────────┬──────────────┬────────────┐
│ customerid ┆ name        ┆ phone        ┆ birthdate  │
│ ---        ┆ ---         ┆ ---          ┆ ---        │
│ i64        ┆ str         ┆ str          ┆ date       │
╞════════════╪═════════════╪══════════════╪════════════╡
│ 3864       ┆ David Perez ┆ 914-594-5535 ┆ 1979-09-29 │
└────────────┴─────────────┴──────────────┴────────────┘


Wow, that narrows it down to one neighbor with the same horoscope sign!
I don't even need to calculate year of the goat in the speedrun.

At this point I could manually inspect their birthdate, but I can just as easily query it.

In [21]:
# year_of_the_rabbit = [2023, 2011, 1999, 1987, 1975, 1963, 1951]
year_of_the_goat = [1907, 1919, 1931, 1943, 1955, 1967, 1979, 1991, 2003, 2015] # https://en.wikipedia.org/wiki/Goat_(zodiac)

print(
    neighbors_same_star_sign.filter(
        pl.col("birthdate").dt.year().is_in(year_of_the_goat)
    ).select("customerid", "name", "phone")
)

shape: (1, 3)
┌────────────┬─────────────┬──────────────┐
│ customerid ┆ name        ┆ phone        │
│ ---        ┆ ---         ┆ ---          │
│ i64        ┆ str         ┆ str          │
╞════════════╪═════════════╪══════════════╡
│ 3864       ┆ David Perez ┆ 914-594-5535 │
└────────────┴─────────────┴──────────────┘
